In [107]:
'''# 1. Preparação de Dados e Engenharia de Features
**Objetivo:** Preparar os dados brutos para o pipeline de Machine Learning, transformando variáveis e definindo o alvo de previsão.

**Etapas do Notebook:**
1.  **Ingestão:** Carregamento do dataset `Sleep_health_and_lifestyle_dataset.csv`.
2.  **Engenharia de Features:** Criação da variável alvo binária (Bom/Ruim).
3.  **Tratamento de Dados:** Preenchimento de valores nulos em desordens do sono.
4.  **Pré-processamento:** Definição de transformações (Padronização e Codificação) e separação Treino/Teste.
5.  **Persistência:** Salvamento dos artefatos (`.joblib`) para uso no notebook de modelagem.'''

'# 1. Preparação de Dados e Engenharia de Features\n**Objetivo:** Preparar os dados brutos para o pipeline de Machine Learning, transformando variáveis e definindo o alvo de previsão.\n\n**Etapas do Notebook:**\n1.  **Ingestão:** Carregamento do dataset `Sleep_health_and_lifestyle_dataset.csv`.\n2.  **Engenharia de Features:** Criação da variável alvo binária (Bom/Ruim).\n3.  **Tratamento de Dados:** Preenchimento de valores nulos em desordens do sono.\n4.  **Pré-processamento:** Definição de transformações (Padronização e Codificação) e separação Treino/Teste.\n5.  **Persistência:** Salvamento dos artefatos (`.joblib`) para uso no notebook de modelagem.'

In [108]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [109]:
df = pd.read_csv("../data/Sleep_health_and_lifestyle_dataset.csv")
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [110]:
'''1.1 Definição do Target (Variável Alvo)
Transformamos o problema de regressão (nota 1-10) em classificação binária para facilitar a geração de planos de ação:
* **1 (Sono Bom):** Qualidade $\ge 7$
* **0 (Sono Ruim):** Qualidade $< 7$'''

'1.1 Definição do Target (Variável Alvo)\nTransformamos o problema de regressão (nota 1-10) em classificação binária para facilitar a geração de planos de ação:\n* **1 (Sono Bom):** Qualidade $\\ge 7$\n* **0 (Sono Ruim):** Qualidade $< 7$'

In [111]:
df["sleep_quality_bin"] = (df["Quality of Sleep"] >= 7).astype(int)
df["sleep_quality_bin"].value_counts()

sleep_quality_bin
1    257
0    117
Name: count, dtype: int64

In [112]:
'''### 1.2 Limpeza de Dados
A coluna `Sleep Disorder` contém valores nulos (`NaN`) para pessoas saudáveis. Substituímos esses valores pela string "Healthy" para que o modelo interprete corretamente como uma categoria.'''

'### 1.2 Limpeza de Dados\nA coluna `Sleep Disorder` contém valores nulos (`NaN`) para pessoas saudáveis. Substituímos esses valores pela string "Healthy" para que o modelo interprete corretamente como uma categoria.'

In [113]:
df["Sleep Disorder"] = df["Sleep Disorder"].fillna('Healthy')
#Tambem temos a coluna "BMI Category" com valores normais e normal weight, que podem ser tratados como a mesma categoria
df["BMI Category"] = df["BMI Category"].replace("Normal Weight", "Normal")
#Tambem temos a coluna "Blood Pressure" que pode ser dividida em Systolic e Diastolic
bp_split = df["Blood Pressure"].str.split("/", expand=True)
df["Systolic_BP"] = pd.to_numeric(bp_split[0], errors='coerce')
df["Diastolic_BP"] = pd.to_numeric(bp_split[1], errors='coerce')
df = df.drop(columns=["Blood Pressure"])

In [114]:
target = "sleep_quality_bin"

numeric_features = [
    "Age",
    "Sleep Duration",
    "Physical Activity Level",
    "Stress Level",
    "Heart Rate",
    "Daily Steps",
    "Systolic_BP",
    "Diastolic_BP"
]

categorical_features = [
    "Gender",
    "Occupation",
    "BMI Category",
    "Sleep Disorder"
]

X = df[numeric_features + categorical_features]
y = df[target]

In [115]:
'''1.3 Pipeline de Pré-processamento
Definimos como cada tipo de dado será tratado antes de entrar no modelo:
* **Numéricos (Age, Steps, etc):** Aplicamos `StandardScaler` para colocar todos na mesma escala (média 0, desvio 1).
* **Categóricos (Gender, Occupation, etc):** Aplicamos `OneHotEncoder` para transformar texto em colunas binárias (0 ou 1).'''

'1.3 Pipeline de Pré-processamento\nDefinimos como cada tipo de dado será tratado antes de entrar no modelo:\n* **Numéricos (Age, Steps, etc):** Aplicamos `StandardScaler` para colocar todos na mesma escala (média 0, desvio 1).\n* **Categóricos (Gender, Occupation, etc):** Aplicamos `OneHotEncoder` para transformar texto em colunas binárias (0 ou 1).'

In [116]:
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [117]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [118]:
'''### 1.4 Persistência de Artefatos
Salvamos o processador (`preprocessor.joblib`) e os dados divididos (`splits.joblib`) na pasta `artifacts`.
Isso garante que o Notebook 2 utilize exatamente a mesma estrutura de dados definida aqui, evitando erros de compatibilidade.'''

'### 1.4 Persistência de Artefatos\nSalvamos o processador (`preprocessor.joblib`) e os dados divididos (`splits.joblib`) na pasta `artifacts`.\nIsso garante que o Notebook 2 utilize exatamente a mesma estrutura de dados definida aqui, evitando erros de compatibilidade.'

In [119]:
joblib.dump(preprocessor, "../artifacts/preprocessor.joblib")
joblib.dump((X_train, X_test, y_train, y_test), "../artifacts/splits.joblib")

['../artifacts/splits.joblib']